# My First Logit Model
This tutorial will give you through the process of creating and estimating a discrete choice model using the Python package Biogeme. Thank you to Professor Michel Bierlaire of EPFL for making his data, code, and examples available for public use.

Your first task is to install Python on your computer for use of the Biogeme package. This can be downloaded as an executable file from the Python website for Windows and Mac machines (Linux comes preloaded with Python). You will need to install Python version 3.7+ for the easiest installation of Biogeme.

# Basic Biogeme Script
The first step is to import the necessary packages: Pandas, Biogeme, and headers.

In [7]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta, Variable, Derive

The next step is to import the estimation data. Biogeme was recently updated to use the excellent package Pandas for data preparation. I import the swissmetro data as a Pandas table, then add it as a database within the internal Biogeme code structure. We can view the first few rows of the data using the command: ```database.data.head()```

In [11]:
pandas = pd.read_csv("./data/swissmetro.dat", sep="\t")
database = db.Database("swissmetro",pandas)

PURPOSE = Variable('PURPOSE')
CHOICE = Variable('CHOICE')
GA = Variable('GA')
TRAIN_CO = Variable('TRAIN_CO')
CAR_AV = Variable('CAR_AV')
SP = Variable('SP')
TRAIN_AV = Variable('TRAIN_AV')
TRAIN_TT = Variable('TRAIN_TT')
SM_TT = Variable('SM_TT')
CAR_TT = Variable('CAR_TT')
CAR_CO = Variable('CAR_CO')
SM_CO = Variable('SM_CO')
SM_AV = Variable('SM_AV')

database.data.head()

,GROUP,SURVEY,SP,ID,PURPOSE,FIRST,TICKET,WHO,LUGGAGE,AGE,...,TRAIN_TT,TRAIN_CO,TRAIN_HE,SM_TT,SM_CO,SM_HE,SM_SEATS,CAR_TT,CAR_CO,CHOICE
0,2,0,1,1,1,0,1,1,0,3,...,112,48,120,63,52,20,0,117,65,2
1,2,0,1,1,1,0,1,1,0,3,...,103,48,30,60,49,10,0,117,84,2
2,2,0,1,1,1,0,1,1,0,3,...,130,48,60,67,58,30,0,117,52,2
3,2,0,1,1,1,0,1,1,0,3,...,103,40,30,63,52,20,0,72,52,2
4,2,0,1,1,1,0,1,1,0,3,...,130,36,60,63,42,20,0,90,84,2


Removing some observations can be done directly using pandas.

In [12]:
print("Size of df before removals: ", database.data.shape)
remove = (((database.data.PURPOSE != 1) & (database.data.PURPOSE != 3)) | (database.data.CHOICE == 0))
database.data.drop(database.data[remove].index,inplace=True)
print("Size of df after removals: ", database.data.shape)

Size of df before removals:  (10728, 28)
Size of df after removals:  (6768, 28)


We can then define the names and starting values for the model parameters. There are 5 parameters defined for the base Swiss Metro model. Biogeme uses a constrained optimization to reduce the search space. The Beta function is called as follows:
```python
Beta(parameter name AS str, initial value AS int, lower bound AS int, upper bound AS int, fixed AS int)
```
If there are no bounds on the parameter search space, the use may enter: ```None```. A value of ```0``` for the ```fixed``` parameter will allow Biogeme to estimate the parameter. In this case, we have only fixed the ASC for the swiss metro mode, to maintain model indentification.

In [13]:
ASC_CAR = Beta('ASC_CAR',0,None,None,0)
ASC_TRAIN = Beta('ASC_TRAIN',0,None,None,0)
ASC_SM = Beta('ASC_SM',0,None,None,1)
B_TIME = Beta('B_TIME',0,None,None,0)
B_COST = Beta('B_COST',0,None,None,0)

We can then define the corresponding variables used in the model. Biogeme has an internal set of functions that are used in the definition of variables. The user can scale a variable value or modify its value using boolean logic.

In [15]:
# Definition of new variables
SM_COST = SM_CO * (GA == 0)
TRAIN_COST = TRAIN_CO * (GA == 0)
CAR_AV_SP = CAR_AV * (SP != 0)
TRAIN_AV_SP = TRAIN_AV * (SP != 0)
TRAIN_TT_SCALED = TRAIN_TT / 100
TRAIN_COST_SCALED = TRAIN_COST / 100
SM_TT_SCALED = SM_TT / 100
SM_COST_SCALED = SM_COST / 100
CAR_TT_SCALED = CAR_TT / 100
CAR_CO_SCALED = CAR_CO / 100

The next step is to define the utility function for each alternative mode.

In [16]:
V1 = ASC_TRAIN + \
     B_TIME * TRAIN_TT_SCALED + \
     B_COST * TRAIN_COST_SCALED
V2 = ASC_SM + \
     B_TIME * SM_TT_SCALED + \
     B_COST * SM_COST_SCALED
V3 = ASC_CAR + \
     B_TIME * CAR_TT_SCALED + \
     B_COST * CAR_CO_SCALED

Biogeme has a variety of built-in discrete choice models, which the user can leverage. However, it also allows the user to define their own likelihood function if it is not available in the built-in model library. In this case, we use the built-in ```bioLogLogit``` function to estimate the model.

In [18]:
# Associate utility functions with the numbering of alternatives
V = {1: V1,
     2: V2,
     3: V3}

# Associate the availability conditions with the alternatives

av = {1: TRAIN_AV_SP,
      2: SM_AV,
      3: CAR_AV_SP}

# Definition of the model. This is the contribution of each
# observation to the log likelihood function.
logprob = models.loglogit(V, av, CHOICE)

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = '01logit'

# Calculate the null log likelihood for reporting.
biogeme.calculateNullLoglikelihood(av)

# Estimate the parameters
results = biogeme.estimate()

[09:14:04] < Warning >   Cannot read file __01logit.iter. Statement is ignored.


The results of the model estimation can then be printed to the terminal.

In [19]:
# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_CAR=	-0.155
ASC_TRAIN=	-0.701
B_COST=	-1.08
B_TIME=	-1.28
              Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CAR   -0.154633      0.058163    -2.658590      0.007847
ASC_TRAIN -0.701187      0.082562    -8.492857      0.000000
B_COST    -1.083790      0.068225   -15.885521      0.000000
B_TIME    -1.277859      0.104254   -12.257120      0.000000


Biogeme also includes functionalities to compute elasticties for model variables. The elasticity can be calculated directly by the user, but Biogeme also includes functions for calculating derivatives for the user.

In [32]:
# The choice model is a logit, with availability conditions
prob1 = models.logit(V, av, 1)
prob2 = models.logit(V, av, 2)
prob3 = models.logit(V, av, 3)

ASC_CAR = Beta('ASC_CAR',betas['ASC_CAR'],None,None,0)
ASC_TRAIN = Beta('ASC_TRAIN',betas['ASC_TRAIN'],None,None,0)
B_TIME = Beta('B_TIME',betas['B_TIME'],None,None,0)
B_COST = Beta('B_COST',betas['B_COST'],None,None,0)

We can then calculate the set of elasticities.

Elasticities can be computed. We illustrate below two formulas. Check in the output file that they produce the same result.

First, the general definition of elasticities. This illustrates the use of the Derive expression, and can be used with any model, however complicated it is. Note the quotes in the Derive operator.

In [33]:
genelas1 = Derive(prob1,'TRAIN_TT') * TRAIN_TT / prob1
genelas2 = Derive(prob2,'SM_TT') * SM_TT / prob2
genelas3 = Derive(prob3,'CAR_TT') * CAR_TT / prob3

print(genelas1)

((Derive({self.child}, "{self.elementName}") * TRAIN_TT) / exp(_bioLogLogit[choice=`1.0`]U=(1:((ASC_TRAIN(init=-0.5119503295055718) + (B_TIME(init=-0.8986643929353354) * (TRAIN_TT / `100.0`))) + (B_COST(init=-0.856666992066188) * ((TRAIN_CO * (GA == `0.0`)) / `100.0`))), 2:((ASC_SM(fixed=0) + (B_TIME(init=-0.8986643929353354) * (SM_TT / `100.0`))) + (B_COST(init=-0.856666992066188) * ((SM_CO * (GA == `0.0`)) / `100.0`))), 3:((ASC_CAR(init=-0.16715433097383006) + (B_TIME(init=-0.8986643929353354) * (CAR_TT / `100.0`))) + (B_COST(init=-0.856666992066188) * (CAR_CO / `100.0`))))av=(1:(TRAIN_AV * (SP != `0.0`)), 2:SM_AV, 3:(CAR_AV * (SP != `0.0`)))))


Second, the elasticity of logit models. See Ben-Akiva and Lerman for the formula.

We can then complete the process using the .simulate() function. This will calculate the probabilities and elasticities for each observation. The Pandas .describe() function is useful to print the statistics for each of these calculated values (i.e. mean, minimum, maximum, standard deviation, percentiles).

In [34]:
logitelas1 = TRAIN_AV_SP * (1.0 - prob1) * TRAIN_TT_SCALED * B_TIME
logitelas2 = SM_AV * (1.0 - prob2) * SM_TT_SCALED * B_TIME
logitelas3 = CAR_AV_SP * (1.0 - prob3) * CAR_TT_SCALED * B_TIME

simulate = {'Prob. train': prob1,
            'Prob. Swissmetro': prob2,
            'Prob. car':prob3,
            'logit elas. 1':logitelas1,
            'generic elas. 1':genelas1,
            'logit elas. 2':logitelas2,
            'generic elas. 2':genelas2,
            'logit elas. 3':logitelas3,
            'generic elas. 3':genelas3}

biogeme  = bio.BIOGEME(database,simulate)
biogeme.modelName = "01logit_simul"
results = biogeme.simulate()
print("Results=",results.describe())

[10:59:10] < Warning >   The chosen alternative [`3.0`] is not available for the following observations (rownumber[choice]): 9[3.0]-10[3.0]-11[3.0]-12[3.0]-13[3.0]-14[3.0]-15[3.0]-16[3.0]-17[3.0]-36[3.0]-37[3.0]-38[3.0]-39[3....
The chosen alternative [`3.0`] is not available for the following observations (rownumber[choice]): 9[3.0]-10[3.0]-11[3.0]-12[3.0]-13[3.0]-14[3.0]-15[3.0]-16[3.0]-17[3.0]-36[3.0]-37[3.0]-38[3.0]-39[3....
The chosen alternative [`3.0`] is not available for the following observations (rownumber[choice]): 9[3.0]-10[3.0]-11[3.0]-12[3.0]-13[3.0]-14[3.0]-15[3.0]-16[3.0]-17[3.0]-36[3.0]-37[3.0]-38[3.0]-39[3....
The chosen alternative [`3.0`] is not available for the following observations (rownumber[choice]): 9[3.0]-10[3.0]-11[3.0]-12[3.0]-13[3.0]-14[3.0]-15[3.0]-16[3.0]-17[3.0]-36[3.0]-37[3.0]-38[3.0]-39[3....
[10:59:10] < Warning >   The chosen alternative [`3.0`] is not available for the following observations (rownumber[choice]): 9[3.0]-10[3.0]-11[3.0]-12[3.0]-13[

# Nested Logit Model
It is straightforward to extend the logit model to a nested specification in Biogeme. We explore a structure with existing modes (i.e. car and train) in one nest and proposed modes (i.e. swiss metro) in a separate nest. We need to define a nesting coefficient for one of the nests, with the other assumed fixed at 1.0.

In [35]:
MU = Beta('MU',2.05,1,10,0)

The utility functions are specified in the same way as for a simple multinomial logit model.

In [36]:
V1 = ASC_TRAIN + \
     B_TIME * TRAIN_TT_SCALED + \
     B_COST * TRAIN_COST_SCALED
V2 = ASC_SM + \
     B_TIME * SM_TT_SCALED + \
     B_COST * SM_COST_SCALED
V3 = ASC_CAR + \
     B_TIME * CAR_TT_SCALED + \
     B_COST * CAR_CO_SCALED

# Associate utility functions with the numbering of alternatives
V = {1: V1,
     2: V2,
     3: V3}

# Associate the availability conditions with the alternatives (un-comment if running as new file)
# CAR_AV_SP =  DefineVariable('CAR_AV_SP',CAR_AV  * (  SP   !=  0  ),database)
# TRAIN_AV_SP =  DefineVariable('TRAIN_AV_SP',TRAIN_AV  * (  SP   !=  0  ),database)

av = {1: TRAIN_AV_SP,
      2: SM_AV,
      3: CAR_AV_SP}

Biogeme requires some additional details to define the nested logit models. We must define the two nests, the scale parameter associated with each nest, and the alternatives in each nest. 

In [25]:
#Definition of nests:
# 1: nests parameter
# 2: list of alternatives
existing = MU , [1,3]
future = 1.0 , [2]
nests = existing,future

The final component of the nested logit code is, again, similar to that for a multinomial logit model. The difference is that we now use the .lognested() function, which requires an additional input parameter specifying the nesting structure.

In [27]:
# The choice model is a nested logit, with availability conditions
logprob = models.lognested(V,av,nests,CHOICE)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "02nested"
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

              Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CAR   -0.167154      0.054529    -3.065434  2.173545e-03
ASC_TRAIN -0.511950      0.079114    -6.471085  9.730172e-11
B_COST    -0.856667      0.060034   -14.269615  0.000000e+00
B_TIME    -0.898664      0.107111    -8.390034  0.000000e+00
MU         2.054036      0.164193    12.509849  0.000000e+00


In [30]:
from biogeme.tools import calculate_correlation
corr = calculate_correlation(
    nests, results, alternative_names={1: 'Train', 2: 'Swissmetro', 3: 'Car'}
)
print(corr)

                 Car  Swissmetro     Train
Car         1.000000         0.0  0.762981
Swissmetro  0.000000         1.0  0.000000
Train       0.762981         0.0  1.000000
